In [1]:
push!(LOAD_PATH, "../")
using MyVQC

In [16]:
using LinearAlgebra

# NOTES: this canonical form will be left-normalized
function tsvd_canonical(a::AbstractArray{𝕋, ℕ}) where {𝕋, ℕ}
    a = Number.(a)
    sizeA = size(a)
    lsize = [sizeA[1], sizeA[2]]
    rsize = sizeA[3]
    F = svd(reshape(a, prod(lsize), rsize))
    u = F.U
    vt = Diagonal(F.S) * F.Vt # absorbed s to vt
    # I think this procedure don't need to be transpose
    #v = transpose(vt)
    
    # reshape to original order
    dim = length(F.S)
    return (reshape(u, lsize..., dim), vt)
end

tsvd_canonical (generic function with 1 method)

In [17]:
a = reshape(rand(10), 1, 2, 5)
u, vt = tsvd_canonical(a)
println(size(u), size(vt))

(1, 2, 2)(2, 5)


In [29]:
function transform_canonical!(sys::MPS)
    for i in 1:(length(sys.data)-1)
        si, vt = tsvd_canonical(sys.data[i])
        sys.data[i] = si
        sys.data[i+1] = contract(vt, sys.data[i+1], (2,), (1,))
    end
end

transform_canonical! (generic function with 1 method)

In [35]:
# test canonical form for EPR state
mps = initmps([0,0,0])
circuit = initcircuit()
push!(circuit, hadamard(1))
push!(circuit, cnot(1,3))
apply!(mps, circuit)
transform_canonical!(mps)
projection_all(mps)


projection for |ϕ>=|000> == 0.7071067811865472
projection for |ϕ>=|001> == 0.0
projection for |ϕ>=|010> == 0.0
projection for |ϕ>=|011> == 0.0
projection for |ϕ>=|100> == 0.0
projection for |ϕ>=|101> == 0.7071067811865472
projection for |ϕ>=|110> == 0.0
projection for |ϕ>=|111> == 0.0


In [50]:
# test canonical form for quantum teleportation
mps = initmps([undef,0,0])
qtelecircuit = initcircuit()
println("probability for #1 qubit in |0> : ", mps.data[1][1,1,1]^2)

# prepare EPR pair
push!(qtelecircuit, hadamard(2))
push!(qtelecircuit, cnot(2,3))
# quantum teleportation
push!(qtelecircuit, cnot(1,2))
push!(qtelecircuit, hadamard(1))
apply!(mps, qtelecircuit)
M₁ = measure!(mps, 1)
M₂ = measure!(mps, 2)
M₂ == 1 && 𝑿!(mps, 3)
M₁ == 1 && 𝒁!(mps, 3)

# measure the original mps system and saved for canonical form
mps_saved = deepcopy(mps)
measure!(mps, 3, true)
transform_canonical!(mps_saved)
measure!(mps_saved, 3, true)

# NOTES: See the print

probability for #1 qubit in |0> : 0.43173225207668126
probability for |0> : 0.4317322520766812
probability for |0> : 0.4317322520766815


0